#### Create a REST API routing map for grants

Just to point out a few features:

- the application should accept POST data to add to the grants variable,
- set routes to get the data about grants by id via GET method,
- set routes to get the list of grants via GET method,
- set routes to delete a grant by id via DELETE method.

Use postman or curl to check if the application fullfil the requirements

In [ ]:
from flask import Flask, request, make_response
app = Flask(__name__)

grants = []

@app.route('/add_grant', methods=['POST'])
def add_grant():
    grant = {"name": request.values["name"],
             "description": request.values["description"],
             "price": request.values["price"]}
    try:
        grants.append(grant)
        response = 'Grant ' + str(grant["name"]) + " is added"
    except:
        response = 'Grant ' + str(grant["name"]) + " is NOT added"
        return make_response(response)
    return make_response(response)


@app.route('/delete_grant', methods=["DELETE"])
def delete_grant():
    grant_id = int(request.values["grant_id"])
    try:
        del grants[grant_id]
        response = 'Grant ' + str(grant_id) + " has been deleted"
    except:
        response = 'Grant ' + str(grant_id) + " doesn't exist"
        return make_response(response)
    return make_response(response)


@app.route('/grants', methods=["GET"])
def grants_list():
    return make_response(str(grants))


@app.route('/grant/<int:id>', methods=["GET"])
def grant(id):
    return make_response(str(grant[id]))


@app.route('/', methods=["GET"])
def index():
    user_agent = request.headers.get('User-Agent')
    return '<p>Your browser is {}</p>'.format(user_agent)

if __name__ == '__main__':
    app.run(port=7001)

!curl -d "param1=value1&param2=value2" -X POST http://localhost:500/<route>

#### Convert previous exercise and add JSON responses and requests

The goal is to response with a JSON to any GET request. Interpret a JSON POST request and interpret the payload. Please keep in mind to set the content type for each GET and POST request. It should be set to application/json. Each POST should return with an error_code response:

- 202 accepted,
- 403 forbidden.

In [ ]:
from flask import Flask, request, make_response
import json
app = Flask(__name__)

grants = []


@app.route('/add_grant', methods=['POST'])
def add_grant():
    grant_json = json.loads(request.get_json())
    try:
        grants.append(grant_json)
        response = {"message":'Grant ' + str(grant_json["name"]) + " is added", "error_code" : 200}
    except:
        response = {"message": 'Grant ' + str(grant_json["name"]) + " is NOT added", "error_code": 500}
        return make_response(json.dumps(response))
    return make_response(json.dumps(response))


@app.route('/delete_grant', methods=["DELETE"])
def delete_grant():
    grant_json = request.get_json()
    grant_id = int(grant_json["grant_id"])
    try:
        del grants[grant_id]
        response = {"message":'Grant ' + str(grant_id) + " has been deleted", "error_code" : 200}
    except:
        response = {"message": 'Grant ' + str(grant_id) + " doesn't exist", "error_code": 500}
        return make_response(json.dumps(response))
    return make_response(json.dumps(response))


@app.route('/grants', methods=["GET"])
def grants_list():
    return make_response(json.dumps(grants))


@app.route('/grant/<int:id>', methods=["GET"])
def grant(id):
    return make_response(json.dumps(grant[id]))


@app.route('/', methods=["GET"])
def index():
    user_agent = request.headers.get('User-Agent')
    return '<p>Your browser is {}</p>'.format(user_agent)

if __name__ == '__main__':
    app.run(port=7001)

#### Handle the upload files for images

Change the code to accept only image files. You can use curl to test your solution:

In [ ]:
from flask import Flask
from flask import request
from flask_uploads import UploadSet, configure_uploads, IMAGES, patch_request_class


app = Flask('Upload example')
app.config['UPLOADS_DEFAULT_DEST'] = '/home/codete/workshop/'

pdfs = UploadSet('files', IMAGES)
configure_uploads(app, pdfs)
patch_request_class(app)  # set maximum file size, default is 16MB

@app.route('/upload', methods=['POST'])
def upload_file():
    filename = pdfs.save(request.files['file'])
    file_url = pdfs.url(filename)
    return "Thank you: " + str(file_url)

if __name__ == '__main__':
    app.run(host="0.0.0.0")

In [ ]:
!curl -F 'file=@<path>/<file>' -X POST http://127.0.0.1:5000/upload